In [18]:
#add environment variables
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

#import crime dataset with cypher queries

movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/ungodlike/LondonCrime/main/LondonCrime.csv' as csv
with csv as crimecsv
where crimecsv.`Location` is not null
merge (p:Place {name: crimecsv.`Location`})
with crimecsv, split(crimecsv.`Month`, "-") AS yearMonth
where yearMonth[0] is not null
merge (y:Year {value: toFloat(yearMonth[0])})
with crimecsv, split(crimecsv.`Month`, "-") AS yearMonth
where yearMonth[1] is not null
merge (m:Month {value: toFloat(yearMonth[1])})
with crimecsv
where crimecsv.`Crime type` is not null
merge (ctg:Category {name: crimecsv.`Crime type`})
with crimecsv
where crimecsv.`Latitude` is not null
merge (c:Case {ref: case when crimecsv.`Crime ID` is null then '' else crimecsv.`Crime ID` end ,lat: toFloat(crimecsv.`Latitude`), lon: toFloat(crimecsv.`Longitude`), outcome: case when crimecsv.`Last outcome category` is null then 'n/a' else crimecsv.`Last outcome category` end })

with crimecsv, split(crimecsv.`Month`, "-") AS yearMonth
match (xc:Case {ref: case when crimecsv.`Crime ID` is null then '' else crimecsv.`Crime ID` end ,lat: toFloat(crimecsv.`Latitude`), lon: toFloat(crimecsv.`Longitude`), outcome: case when crimecsv.`Last outcome category` is null then 'n/a' else crimecsv.`Last outcome category` end }),
(xy:Year {value: toFloat(yearMonth[0])}),
(xm:Month {value: toFloat(yearMonth[1])}),
(xp:Place {name: crimecsv.`Location`}),
(xctg:Category {name: crimecsv.`Crime type`})

create (xm)-[:YEAR_OF]->(xy),
(xc)-[:HAPPEN_IN]->(xm),
(xc)-[:TYPE_OF]->(xctg),
(xc)-[:AT]->(xp);
"""

graph.query(movies_query)

In [20]:
graph.refresh_schema() #to refresh schema on changes
print(graph.schema)

Node properties:
Place {name: STRING}
Year {value: FLOAT}
Month {value: FLOAT}
Category {name: STRING}
Case {outcome: STRING, ref: STRING, lat: FLOAT, lon: FLOAT}
Relationship properties:

The relationships:
(:Month)-[:YEAR_OF]->(:Year)
(:Case)-[:TYPE_OF]->(:Category)
(:Case)-[:HAPPEN_IN]->(:Month)
(:Case)-[:AT]->(:Place)


In [21]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
#choose llm and create graph query chain
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)


In [23]:
#obtain query and produce output
user_query = input("Enter your query: ")
response = chain.invoke({"query": user_query})
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Case {ref: 'ab9a10f63343d679961a09e9c2305fb4a9b871f05f64e1a3368ceb78cab973d5'})-[:TYPE_OF]->(c:Category)
RETURN c.name;
Full Context:
[{'c.name': 'Violence and sexual offences'}, {'c.name': 'Violence and sexual offences'}]

> Finished chain.
{'query': 'In what category was case ab9a10f63343d679961a09e9c2305fb4a9b871f05f64e1a3368ceb78cab973d5', 'result': 'The case ab9a10f63343d679961a09e9c2305fb4a9b871f05f64e1a3368ceb78cab973d5 was categorized under Violence and sexual offences.'}
